In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json


HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

#Reading the csv file for generating url
column_names = ["", "id", "Asin","country"]
df = pd.read_csv("F:\data_cisco\Scraping.csv", names=column_names)
Asin = df.Asin.to_list()
country = df.country.to_list()

#Dictonary that will contain the extracted data
dict={}

#This function will extract the required data from given url
def f(url,i):
    page = requests.get(url, headers=HEADERS)
    if(page.status_code==404):
        print(str(url)+" not available")
        dict[i]={i:{url:"not available"}}
    else:    
        soup = BeautifulSoup(page.content, features="lxml")
           
        #Product Title
        title = soup.find(id='productTitle').get_text().strip()
       
        #Image URL
        img=soup.find('div',{"id":"imgTagWrapperId"})
        if img is None:
            img=soup.find('div',{"id":"img-canvas"})
        imgurl=img.find('img').attrs['src']
    
        #Price of Product
        price = soup.find("span", class_="a-offscreen").get_text().strip()
    
        #Product Details
        desc = soup.find("div", id="feature-bullets")
        if desc is None:
             desc = soup.find("div", id="detailBullets_feature_div")
        
        desc=desc.get_text().replace('About this item','').strip()
    
    
        dict[i]={i:{'Title':title, 'ImageURL':imgurl, 'Price':price, 'Product Details':desc}}
        
#This function will generate the url
for i in range(1,1000):
    ur = "https://www.amazon."+str(country[i])+"/dp/"+str(Asin[i])
    f(ur,i)
    
#Creating Json file   
json_object = json.dumps(dict, indent = 4)
with open("sample10.json", "w") as outfile:
    outfile.write(json_object)